In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 14.0 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
dt[3000].value_counts()

2    49030
0    30515
4    19065
3     9095
1     9013
Name: 3000, dtype: int64

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_481.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_481_trial_3.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118693,0.003997,0.004450,0.019831,0.028427,0.027974,0.005354,-0.015456,-0.011384,0.017117,0.010783,...,0.001735,-0.032194,-0.073814,-0.102315,-0.091005,-0.076076,-0.055719,-0.005955,0.046522,4
118694,-0.057506,-0.007102,0.013150,0.029351,0.032502,0.050503,0.029351,0.045103,0.082906,0.074355,...,0.189114,0.132410,0.136010,0.131060,0.136010,0.144561,0.173813,0.172463,0.131510,2
118695,-0.043052,-0.035361,-0.011384,-0.004146,-0.014551,-0.032194,-0.029027,-0.030837,-0.040790,-0.036266,...,0.037474,0.008069,-0.022694,-0.051647,-0.032646,-0.035361,-0.037623,-0.055719,-0.073814,2
118696,-0.006202,-0.000351,0.003699,-0.012502,-0.079557,-0.057506,0.011350,-0.030053,-0.048955,0.048253,...,0.025751,0.022151,0.038802,0.039252,0.010900,0.013600,-0.029603,-0.034104,-0.066056,0


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 16.0 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
x_dt.shape

(118698, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118693,0.003997,0.004450,0.019831,0.028427,0.027974,0.005354,-0.015456,-0.011384,0.017117,0.010783,...,0.041546,0.001735,-0.032194,-0.073814,-0.102315,-0.091005,-0.076076,-0.055719,-0.005955,0.046522
118694,-0.057506,-0.007102,0.013150,0.029351,0.032502,0.050503,0.029351,0.045103,0.082906,0.074355,...,0.200365,0.189114,0.132410,0.136010,0.131060,0.136010,0.144561,0.173813,0.172463,0.131510
118695,-0.043052,-0.035361,-0.011384,-0.004146,-0.014551,-0.032194,-0.029027,-0.030837,-0.040790,-0.036266,...,0.017117,0.037474,0.008069,-0.022694,-0.051647,-0.032646,-0.035361,-0.037623,-0.055719,-0.073814
118696,-0.006202,-0.000351,0.003699,-0.012502,-0.079557,-0.057506,0.011350,-0.030053,-0.048955,0.048253,...,0.033852,0.025751,0.022151,0.038802,0.039252,0.010900,0.013600,-0.029603,-0.034104,-0.066056


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(118698,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[-0.04305152],
        [-0.03536085],
        [-0.01138404],
        ...,
        [-0.03762281],
        [-0.05571852],
        [-0.07381422]],

       [[-0.00620155],
        [-0.00035108],
        [ 0.00369925],
        ...,
        [-0.02960344],
        [-0.03410381],
        [-0.06605639]],

       [[ 0.03430173],
        [ 0.05230318],
        [ 0.06760442],
        ...,
        [ 0.1661624 ],
        [ 0.25121929],
        [ 0.30297347]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-p9fu2t0b
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-p9fu2t0b
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=5c109b020c38c019a8a565cceb41b8dc3a73ed46e81f8e8d9f228b08be64adb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-0f4lz2cj/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt
optimizer=Adam(lr=1e-4)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
      model=model_b(verbose=VBS)
      optimizer = Adam(lr=1e-4)
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
      optimizer = Adam(lr=1e-4)
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            x_dt_rshp, y_dt_,
            epochs=hp.Choice("epochs", [40, 50, 60, 70])
        )

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective="loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    #directory="my_dir",
    #project_name="tune_hypermodel",
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tuner.search(x_dt_rshp, y_dt_)

Trial 3 Complete [00h 35m 37s]
loss: 0.08628439903259277

Best loss So Far: 0.05698661878705025
Total elapsed time: 02h 13m 24s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
epochs: 70
Score: 0.05698661878705025
Trial summary
Hyperparameters:
epochs: 40
Score: 0.08628439903259277
Trial summary
Hyperparameters:
epochs: 40
Score: 0.08674326911568642


tuner.search() O/P:

Trial 1:

Trial 3 Complete [01h 05m 12s]

loss: 0.06334519572556019

Best loss So Far: 0.055410414934158325

Total elapsed time: 03h 07m 15s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() O/P:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7fea554ea1d0>

Trial summary

Hyperparameters:

epochs: 70

Score: 0.055410414934158325

Trial summary

Hyperparameters:

epochs: 60

Score: 0.06334519572556019

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08637035638093948

tuner.search() O/P:

Trial 2:

Trial 3 Complete [00h 41m 57s]

loss: 0.08599428460001945

Best loss So Far: 0.05632760375738144

Total elapsed time: 02h 34m 48s

INFO:tensorflow:Oracle triggered exit

tuner.results_summary() O/P:

Results summary

Results in ./untitled_project

Showing 10 best trials

<keras_tuner.engine.objective.Objective object at 0x7f75ff41a4d0>

Trial summary

Hyperparameters:

epochs: 70

Score: 0.05632760375738144

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08599428460001945

Trial summary

Hyperparameters:

epochs: 40

Score: 0.08637384697794914


In [ ]:
teacher_40_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_40_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Training the teacher model for 40 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_40_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_40_epochs_dir = os.path.dirname(checkpoint_teacher_model_40_epochs)

In [ ]:
cp_callback_teacher_model_40_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_40_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=40)

In [ ]:
teacher_40_epochs.fit(x_dt_rshp, y_dt_, epochs=40, callbacks=[cp_callback_teacher_model_40_epochs])

Epoch 1/40
3710/3710 [==============================] - 47s 9ms/step - loss: 0.6631 - accuracy: 0.7323
Epoch 2/40
3710/3710 [==============================] - 32s 9ms/step - loss: 0.3753 - accuracy: 0.8529
Epoch 3/40
3710/3710 [==============================] - 32s 9ms/step - loss: 0.3149 - accuracy: 0.8759
Epoch 4/40
3710/3710 [==============================] - 34s 9ms/step - loss: 0.2753 - accuracy: 0.8917
Epoch 5/40
3710/3710 [==============================] - 34s 9ms/step - loss: 0.2439 - accuracy: 0.9034
Epoch 6/40
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2240 - accuracy: 0.9115
Epoch 7/40
3710/3710 [==============================] - 32s 9ms/step - loss: 0.2078 - accuracy: 0.9182
Epoch 8/40
3710/3710 [==============================] - 32s 9ms/step - loss: 0.1935 - accuracy: 0.9239
Epoch 9/40
3710/3710 [==============================] - 32s 9ms/step - loss: 0.1842 - accuracy: 0.9266
Epoch 10/40
3710/3710 [==============================] - 32s 9ms/step - l

In [ ]:
latest_teacher_40_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_40_epochs_dir)

In [ ]:
print(latest_teacher_40_epochs)
print(checkpoint_teacher_model_40_epochs)

/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt
/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt


In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(496, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.026201,0.021701,0.037902,0.021701,0.022601,-0.004401,0.019901,0.011800,-0.037704,0.031151,...,0.049153,0.068504,0.052753,0.062654,0.027101,-0.001251,-0.031404,0.004599,-0.030053,-0.027803
1,-0.102959,-0.104309,-0.099359,-0.129061,-0.121861,-0.101159,-0.045355,-0.039954,-0.031404,-0.017902,...,0.008200,0.038802,0.087406,0.138260,0.129709,0.124309,0.086506,0.072105,0.023501,-0.022853
2,-0.122673,-0.129459,-0.115887,-0.080600,-0.076529,-0.056171,-0.057076,-0.059790,-0.055719,-0.036718,...,0.084071,0.084071,0.064618,0.064166,0.050594,0.028427,0.027974,0.044713,0.058284,0.060999
3,-0.088108,-0.084508,-0.039504,-0.067857,-0.053005,-0.080908,-0.124561,-0.005752,-0.089008,-0.053455,...,-0.087208,-0.067407,-0.017002,-0.103409,-0.083608,-0.025553,-0.054805,0.020801,-0.003501,-0.029153
4,0.136010,0.146811,0.200815,0.159862,0.163912,0.123859,0.130159,0.105407,0.109008,0.039702,...,0.070305,0.091456,0.050503,0.057704,0.041502,0.019901,0.015850,0.043302,0.035652,0.047353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,-0.056606,-0.027803,-0.049405,-0.039504,-0.010252,0.011350,0.004599,0.000099,-0.017452,-0.039504,...,-0.059756,0.022601,-0.032304,-0.155614,-0.108810,-0.101159,-0.018353,-0.080458,-0.070107,-0.024203
492,-0.022241,-0.010932,-0.014551,-0.015003,-0.006860,-0.017718,-0.023146,-0.025408,-0.015908,-0.029480,...,-0.045766,-0.015456,0.015307,0.024807,0.003545,-0.018170,-0.016813,0.002188,-0.003693,-0.000979
493,0.021188,0.054665,0.073213,0.074118,0.048332,0.034760,0.027974,0.044713,0.057832,0.051951,...,0.320672,0.275885,0.258242,0.268647,0.303029,0.313886,0.315243,0.309815,0.301219,0.291267
494,0.084976,0.079547,0.075928,0.056022,0.067785,0.084071,0.134286,0.123881,0.118453,0.083618,...,0.095381,0.103976,0.122977,0.118453,0.092214,0.057832,0.060546,0.058737,0.056475,0.035665


In [ ]:
x_test_dt.shape

(496, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(496,3000,1)

In [ ]:
x_test_dt_rshp

array([[[ 0.02620107],
        [ 0.02170071],
        [ 0.03790202],
        ...,
        [ 0.00459933],
        [-0.03005348],
        [-0.0278033 ]],

       [[-0.10295938],
        [-0.10430949],
        [-0.09935909],
        ...,
        [ 0.07210479],
        [ 0.02350085],
        [-0.0228529 ]],

       [[-0.12267263],
        [-0.12945852],
        [-0.11588674],
        ...,
        [ 0.04471265],
        [ 0.05828443],
        [ 0.06099879]],

       ...,

       [[ 0.02118823],
        [ 0.05466529],
        [ 0.07321339],
        ...,
        [ 0.30981474],
        [ 0.30121928],
        [ 0.29126664]],

       [[ 0.0849756 ],
        [ 0.07954689],
        [ 0.07592774],
        ...,
        [ 0.05873682],
        [ 0.05647486],
        [ 0.0356648 ]],

       [[-0.04490468],
        [-0.02915341],
        [ 0.00324922],
        ...,
        [ 0.07255483],
        [ 0.06715439],
        [ 0.08290567]]])

In [ ]:
accuracy_teacher_model_40_epochs = teacher_40_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

Training the teacher model for 50 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_50_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_50_epochs_dir = os.path.dirname(checkpoint_teacher_model_50_epochs)

In [ ]:
cp_callback_teacher_model_50_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_50_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=50)

In [ ]:
teacher_50_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_50_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_50_epochs.fit(x_dt_rshp, y_dt_, epochs=50, callbacks=[cp_callback_teacher_model_50_epochs])

Epoch 1/50
3710/3710 [==============================] - 44s 8ms/step - loss: 0.6638 - accuracy: 0.7314
Epoch 2/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.3755 - accuracy: 0.8529
Epoch 3/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.3146 - accuracy: 0.8758
Epoch 4/50
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2702 - accuracy: 0.8937
Epoch 5/50
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2399 - accuracy: 0.9057
Epoch 6/50
3710/3710 [==============================] - 28s 8ms/step - loss: 0.2193 - accuracy: 0.9138
Epoch 7/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2052 - accuracy: 0.9188
Epoch 8/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1914 - accuracy: 0.9245
Epoch 9/50
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1838 - accuracy: 0.9272
Epoch 10/50
3710/3710 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_50_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_50_epochs_dir)

In [ ]:
accuracy_teacher_model_50_epochs = teacher_50_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 8ms/step - loss: 0.8527 - accuracy: 0.8185


Training the teacher model for 60 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_60_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_60_epochs_dir = os.path.dirname(checkpoint_teacher_model_60_epochs)

In [ ]:
cp_callback_teacher_model_60_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_60_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
teacher_60_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_60_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_60_epochs.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])

Epoch 1/60
3710/3710 [==============================] - 45s 8ms/step - loss: 0.6640 - accuracy: 0.7311
Epoch 2/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.3760 - accuracy: 0.8523
Epoch 3/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.3155 - accuracy: 0.8757
Epoch 4/60
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2765 - accuracy: 0.8913
Epoch 5/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.2460 - accuracy: 0.9037
Epoch 6/60
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2229 - accuracy: 0.9118
Epoch 7/60
3710/3710 [==============================] - 29s 8ms/step - loss: 0.2081 - accuracy: 0.9180
Epoch 8/60
3710/3710 [==============================] - 30s 8ms/step - loss: 0.1954 - accuracy: 0.9229
Epoch 9/60
3710/3710 [==============================] - 29s 8ms/step - loss: 0.1862 - accuracy: 0.9258
Epoch 10/60
3710/3710 [==============================] - 29s 8ms/step - l

In [ ]:
latest_teacher_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_60_epochs_dir)

In [ ]:
accuracy_teacher_model_60_epochs = teacher_60_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 9ms/step - loss: 0.8504 - accuracy: 0.8206


Training the teacher model for 70 epochs wrt to the transfer set, majority voted labels, x_sub_train, tri-trained labels

In [ ]:
checkpoint_teacher_model_70_epochs = "/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"

In [ ]:
checkpoint_teacher_model_70_epochs_dir = os.path.dirname(checkpoint_teacher_model_70_epochs)

In [ ]:
cp_callback_teacher_model_70_epochs = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_70_epochs,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=70)

In [ ]:
teacher_70_epochs=model_b(verbose=VBS)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
teacher_70_epochs.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
teacher_70_epochs.fit(x_dt_rshp, y_dt_, epochs=70, callbacks=[cp_callback_teacher_model_70_epochs])

Epoch 1/70
3710/3710 [==============================] - 31s 7ms/step - loss: 0.6483 - accuracy: 0.7423
Epoch 2/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.3778 - accuracy: 0.8515
Epoch 3/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.3224 - accuracy: 0.8727
Epoch 4/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.2822 - accuracy: 0.8880
Epoch 5/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.2529 - accuracy: 0.9002
Epoch 6/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.2286 - accuracy: 0.9088
Epoch 7/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.2135 - accuracy: 0.9146
Epoch 8/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.1971 - accuracy: 0.9221
Epoch 9/70
3710/3710 [==============================] - 27s 7ms/step - loss: 0.1873 - accuracy: 0.9252
Epoch 10/70
3710/3710 [==============================] - 27s 7ms/step - l

In [ ]:
latest_teacher_70_epochs=tensorflow.train.latest_checkpoint(checkpoint_teacher_model_70_epochs_dir)

In [ ]:
accuracy_teacher_model_70_epochs = teacher_70_epochs.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 7ms/step - loss: 0.9306 - accuracy: 0.8226


Trial 1

KD using the 50 epochs teacher model, finalized the teacher with 60 epochs, student model: alpha: 0.9, temp: 4

Trial 2

KD using the 40 epochs teacher model, finalized the teacher with 50 epochs, student model: alpha: 0.9, temp: 4

Trial 3

KD using the 40 epochs teacher model, finalized the teacher with 50 epochs, student model: alpha: 0.9, temp: 4

In [ ]:
teacher_model=model_b(verbose=VBS) #40 epochs teacher model 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/40_epochs_keras_way/teacher_40_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 13s 9ms/step - loss: 0.7190 - accuracy: 0.8367


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.83928571, 0.43137255, 0.98245614, 0.69565217, 0.86666667])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.8211025930220244
Recall: 0.7630866488471819
F1 score: 0.7846217931968913
              precision    recall  f1-score   support

           0   0.952703  0.839286  0.892405       168
           1   0.458333  0.431373  0.444444        51
           2   0.796209  0.982456  0.879581       171
           3   0.969697  0.695652  0.810127        46
           4   0.928571  0.866667  0.896552        60

    accuracy                       0.836694       496
   macro avg   0.821103  0.763087  0.784622       496
weighted avg   0.846575  0.836694  0.834794       496



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=50)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/50
3710/3710 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6834 - student_loss: 0.7702 - distillation_loss: 0.0016
Epoch 2/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7622 - student_loss: 0.5729 - distillation_loss: 0.0012
Epoch 3/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7815 - student_loss: 0.5271 - distillation_loss: 0.0011
Epoch 4/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7953 - student_loss: 0.4962 - distillation_loss: 0.0010
Epoch 5/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8062 - student_loss: 0.4707 - distillation_loss: 9.7942e-04
Epoch 6/50
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8209 - student_loss: 0.4419 - distillation_loss: 9.0692e-04
Epoch 7/50
3710/3710 [==============================] - 29s 8ms/step - categorical_acc

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8266 - student_loss: 0.5402


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.95238095, 0.1372549 , 0.97660819, 0.58695652, 0.81666667])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.7875015601769425
Recall: 0.6939734459764073
F1 score: 0.7131405916073124
              precision    recall  f1-score   support

           0   0.883978  0.952381  0.916905       168
           1   0.437500  0.137255  0.208955        51
           2   0.776744  0.976608  0.865285       171
           3   0.964286  0.586957  0.729730        46
           4   0.875000  0.816667  0.844828        60

    accuracy                       0.826613       496
   macro avg   0.787502  0.693973  0.713141       496
weighted avg   0.807462  0.826613  0.800237       496



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.95238095, 0.1372549 , 0.97660819, 0.58695652, 0.81666667])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
#50 epochs teacher model

#alpha=0.9, temp=4

#50 epochs: 82.41
#60 epochs: 84.53
#70 epochs: 84.04

#alpha=0.9, temp=5

#50 epochs: 81.43
#60 epochs: 83.88
#70 epochs: 

#alpha=0.9, temp=3

#50 epochs: 83.88
#60 epochs: 83.06


KD using the 50 epochs teacher model

In [ ]:
teacher_model=model_b(verbose=VBS) #50 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs_keras_way/teacher_50_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 0.8298 - accuracy: 0.8266


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.80952381, 0.50980392, 0.97076023, 0.63043478, 0.88333333])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.8169118068268141
Recall: 0.7607712161905189
F1 score: 0.7783872000830183
              precision    recall  f1-score   support

           0   0.957746  0.809524  0.877419       168
           1   0.448276  0.509804  0.477064        51
           2   0.798077  0.970760  0.875989       171
           3   0.966667  0.630435  0.763158        46
           4   0.913793  0.883333  0.898305        60

    accuracy                       0.826613       496
   macro avg   0.816912  0.760771  0.778387       496
weighted avg   0.845824  0.826613  0.827691       496



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=60)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/60
3710/3710 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6859 - student_loss: 0.7922 - distillation_loss: 0.0026
Epoch 2/60
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7710 - student_loss: 0.5497 - distillation_loss: 0.0019
Epoch 3/60
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7884 - student_loss: 0.5091 - distillation_loss: 0.0018
Epoch 4/60
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8045 - student_loss: 0.4782 - distillation_loss: 0.0016
Epoch 5/60
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8172 - student_loss: 0.4492 - distillation_loss: 0.0015
Epoch 6/60
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.8292 - student_loss: 0.4248 - distillation_loss: 0.0014
Epoch 7/60
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8165 - student_loss: 0.5412


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.92261905, 0.19607843, 0.95906433, 0.65217391, 0.76666667])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.7703115434207872
Recall: 0.6993204772374243
F1 score: 0.7218445361361118
              precision    recall  f1-score   support

           0   0.911765  0.922619  0.917160       168
           1   0.384615  0.196078  0.259740        51
           2   0.759259  0.959064  0.847545       171
           3   0.857143  0.652174  0.740741        46
           4   0.938776  0.766667  0.844037        60

    accuracy                       0.816532       496
   macro avg   0.770312  0.699320  0.721845       496
weighted avg   0.803186  0.816532  0.800355       496



In [ ]:
#60 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 14s 9ms/step - loss: 0.8025 - accuracy: 0.8246


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.82142857, 0.41176471, 0.97076023, 0.65217391, 0.9       ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.7999419508594282
Recall: 0.7512254848545064
F1 score: 0.7677400558380658
              precision    recall  f1-score   support

           0   0.945205  0.821429  0.878981       168
           1   0.437500  0.411765  0.424242        51
           2   0.794258  0.970760  0.873684       171
           3   0.937500  0.652174  0.769231        46
           4   0.885246  0.900000  0.892562        60

    accuracy                       0.824597       496
   macro avg   0.799942  0.751225  0.767740       496
weighted avg   0.832994  0.824597  0.821862       496



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3710/3710 [==============================] - 35s 8ms/step - categorical_accuracy: 0.6796 - student_loss: 0.7805 - distillation_loss: 0.0017
Epoch 2/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accuracy: 0.7584 - student_loss: 0.5717 - distillation_loss: 0.0013
Epoch 3/70
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7798 - student_loss: 0.5245 - distillation_loss: 0.0012
Epoch 4/70
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7922 - student_loss: 0.4970 - distillation_loss: 0.0011
Epoch 5/70
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8047 - student_loss: 0.4729 - distillation_loss: 0.0011
Epoch 6/70
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8176 - student_loss: 0.4462 - distillation_loss: 9.8070e-04
Epoch 7/70
3710/3710 [==============================] - 30s 8ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8327 - student_loss: 0.5140


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.93452381, 0.19607843, 0.97660819, 0.67391304, 0.8       ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.797292093042859
Recall: 0.7162246943018244
F1 score: 0.7350291227255272
              precision    recall  f1-score   support

           0   0.912791  0.934524  0.923529       168
           1   0.526316  0.196078  0.285714        51
           2   0.780374  0.976608  0.867532       171
           3   0.939394  0.673913  0.784810        46
           4   0.827586  0.800000  0.813559        60

    accuracy                       0.832661       496
   macro avg   0.797292  0.716225  0.735029       496
weighted avg   0.819561  0.832661  0.812474       496



In [ ]:
#70 epochs teacher

In [ ]:
teacher_model=model_b(verbose=VBS) #70 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/70_epochs_keras_way/teacher_70_epochs_keras_way_wts_trial_3.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 11s 9ms/step - loss: 0.9306 - accuracy: 0.8226


In [ ]:
preds_teacher_model = teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_teacher_model_fin=np.argmax(preds_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_teacher = confusion_matrix(y_test_dt, preds_teacher_model_fin)
matrix_teacher.diagonal()/matrix_teacher.sum(axis=1)

array([0.81547619, 0.37254902, 0.97660819, 0.67391304, 0.9       ])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_teacher = f1_score(y_test_dt, preds_teacher_model_fin, average="macro")
prec_teacher = precision_score(y_test_dt, preds_teacher_model_fin, average="macro")
recall_teacher = recall_score(y_test_dt, preds_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_teacher)) 
    print("Recall: {}".format(recall_teacher))
    print("F1 score: {}".format(f1_teacher))

    report_teacher = classification_report(y_test_dt, preds_teacher_model_fin, digits=6)
    print(report_teacher)

Precision: 0.7979769291533252
Recall: 0.7460825657494119
F1 score: 0.7663813455270547
              precision    recall  f1-score   support

           0   0.946309  0.839286  0.889590       168
           1   0.416667  0.392157  0.404040        51
           2   0.785714  0.964912  0.866142       171
           3   0.916667  0.717391  0.804878        46
           4   0.924528  0.816667  0.867257        60

    accuracy                       0.822581       496
   macro avg   0.797977  0.746083  0.766381       496
weighted avg   0.831100  0.822581  0.821023       496



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=80)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/80
3710/3710 [==============================] - 34s 8ms/step - categorical_accuracy: 0.6825 - student_loss: 0.7752 - distillation_loss: 0.0017
Epoch 2/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7594 - student_loss: 0.5744 - distillation_loss: 0.0013
Epoch 3/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7765 - student_loss: 0.5356 - distillation_loss: 0.0012
Epoch 4/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.7874 - student_loss: 0.5087 - distillation_loss: 0.0011
Epoch 5/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8002 - student_loss: 0.4843 - distillation_loss: 0.0011
Epoch 6/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0.8113 - student_loss: 0.4598 - distillation_loss: 0.0010
Epoch 7/80
3710/3710 [==============================] - 29s 8ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8347 - student_loss: 0.4912


In [ ]:
preds_student_model = student_model.predict(x_test_dt_rshp)

In [ ]:
preds_student_model_fin=np.argmax(preds_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_student = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix_student.diagonal()/matrix_student.sum(axis=1)

array([0.94642857, 0.17647059, 0.97660819, 0.65217391, 0.81666667])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_student = f1_score(y_test_dt, preds_student_model_fin, average="macro")
prec_student = precision_score(y_test_dt, preds_student_model_fin, average="macro")
recall_student = recall_score(y_test_dt, preds_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_student)) 
    print("Recall: {}".format(recall_student))
    print("F1 score: {}".format(f1_student))

    report_student = classification_report(y_test_dt, preds_student_model_fin, digits=6)
    print(report_student)

Precision: 0.7936604412951678
Recall: 0.7136695853017028
F1 score: 0.7366282707758108
              precision    recall  f1-score   support

           0   0.893258  0.946429  0.919075       168
           1   0.391304  0.176471  0.243243        51
           2   0.791469  0.976608  0.874346       171
           3   0.967742  0.652174  0.779221        46
           4   0.924528  0.816667  0.867257        60

    accuracy                       0.834677       496
   macro avg   0.793660  0.713670  0.736628       496
weighted avg   0.817244  0.834677  0.814925       496



Trying out diff teachers and different students for perspective(the teacher who's epochs had been tuned)



In [ ]:
final_teacher_model=model_b(verbose=VBS) #70 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
final_teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_teacher_model.fit(x_dt_rshp, y_dt_, epochs=70)

Epoch 1/70
3710/3710 [==============================] - 50s 9ms/step - loss: 0.6628 - accuracy: 0.7326
Epoch 2/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.3743 - accuracy: 0.8539
Epoch 3/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.3162 - accuracy: 0.8753
Epoch 4/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2764 - accuracy: 0.8915
Epoch 5/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2461 - accuracy: 0.9030
Epoch 6/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2241 - accuracy: 0.9122
Epoch 7/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.2094 - accuracy: 0.9171
Epoch 8/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.1950 - accuracy: 0.9229
Epoch 9/70
3710/3710 [==============================] - 33s 9ms/step - loss: 0.1858 - accuracy: 0.9264
Epoch 10/70
3710/3710 [==============================] - 32s 9ms/step - l

In [ ]:
accuracy_final_teacher_model = final_teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 1s 9ms/step - loss: 1.0005 - accuracy: 0.8105


In [ ]:
preds_final_teacher_model = final_teacher_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_teacher_model_fin=np.argmax(preds_final_teacher_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_teacher = confusion_matrix(y_test_dt, preds_final_teacher_model_fin)
matrix_final_teacher.diagonal()/matrix_final_teacher.sum(axis=1)

array([0.80357143, 0.43137255, 0.97076023, 0.56521739, 0.88333333])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_teacher = f1_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
prec_final_teacher = precision_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
recall_final_teacher = recall_score(y_test_dt, preds_final_teacher_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_teacher)) 
    print("Recall: {}".format(recall_final_teacher))
    print("F1 score: {}".format(f1_final_teacher))

    report_final_teacher = classification_report(y_test_dt, preds_final_teacher_model_fin, digits=6)
    print(report_final_teacher)

Precision: 0.7900371607604706
Recall: 0.7308509872293693
F1 score: 0.7464656909960764
              precision    recall  f1-score   support

           0   0.957447  0.803571  0.873786       168
           1   0.431373  0.431373  0.431373        51
           2   0.783019  0.970760  0.866841       171
           3   0.962963  0.565217  0.712329        46
           4   0.815385  0.883333  0.848000        60

    accuracy                       0.810484       496
   macro avg   0.790037  0.730851  0.746466       496
weighted avg   0.826546  0.810484  0.807808       496



In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
final_student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=final_student_model, teacher=final_teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=1.0,
    temperature=5,
)

# Distill teacher to student
distiller.fit(x_dt_rshp, y_dt_, epochs=80)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/80
3710/3710 [==============================] - 35s 9ms/step - categorical_accuracy: 0.6719 - student_loss: 0.7993 - distillation_loss: 0.0017
Epoch 2/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accuracy: 0.7579 - student_loss: 0.5803 - distillation_loss: 0.0013
Epoch 3/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accuracy: 0.7743 - student_loss: 0.5414 - distillation_loss: 0.0012
Epoch 4/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accuracy: 0.7910 - student_loss: 0.5063 - distillation_loss: 0.0011
Epoch 5/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accuracy: 0.8057 - student_loss: 0.4731 - distillation_loss: 0.0011
Epoch 6/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accuracy: 0.8191 - student_loss: 0.4453 - distillation_loss: 9.8181e-04
Epoch 7/80
3710/3710 [==============================] - 32s 9ms/step - categorical_accurac

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8246 - student_loss: 0.4820


In [ ]:
preds_final_student_model = final_student_model.predict(x_test_dt_rshp)

In [ ]:
preds_final_student_model_fin=np.argmax(preds_final_student_model,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_final_student = confusion_matrix(y_test_dt, preds_final_student_model_fin)
matrix_final_student.diagonal()/matrix_final_student.sum(axis=1)

array([0.95833333, 0.15686275, 0.93567251, 0.7173913 , 0.78333333])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_final_student = f1_score(y_test_dt, preds_final_student_model_fin, average="macro")
prec_final_student = precision_score(y_test_dt, preds_final_student_model_fin, average="macro")
recall_final_student = recall_score(y_test_dt, preds_final_student_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_final_student)) 
    print("Recall: {}".format(recall_final_student))
    print("F1 score: {}".format(f1_final_student))

    report_final_student = classification_report(y_test_dt, preds_final_student_model_fin, digits=6)
    print(report_final_student)

Precision: 0.7649748023000302
Recall: 0.710318646146483
F1 score: 0.724184112455528
              precision    recall  f1-score   support

           0   0.856383  0.958333  0.904494       168
           1   0.380952  0.156863  0.222222        51
           2   0.816327  0.935673  0.871935       171
           3   0.916667  0.717391  0.804878        46
           4   0.854545  0.783333  0.817391        60

    accuracy                       0.824597       496
   macro avg   0.764975  0.710319  0.724184       496
weighted avg   0.799057  0.824597  0.803341       496



In [ ]:
#40 epochs teacher model
#alpha=0.9, temp=4
#40 epochs: 83.71
#50 epochs: 84.36
#60 epochs: 83.88